In [ ]:
"""
Created on Wed Mar 20 15:07 2024

This script is to make 2D maps of viability limits

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib as mpl
import cmocean
import glob
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

READ DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'


In [ ]:
GL_flux = xr.open_dataset(outputpath_GL + 'all_GL_fluxes_varying_m.nc')

In [ ]:
GL_flux_ag = xr.Dataset()
GL_flux_ag['flux_Gt_ref'] = xr.concat([GL_flux['flux_Gt_ref_m1'].assign_coords({'m': 1}),
                                       GL_flux['flux_Gt_ref_m3'].assign_coords({'m': 3}),
                                       GL_flux['flux_Gt_ref_m5'].assign_coords({'m': 5})], dim='m')
GL_flux_ag['flux_Gt_ABUMIP'] = xr.concat([GL_flux['flux_Gt_ABUMIP_m1'].assign_coords({'m': 1}),
                                       GL_flux['flux_Gt_ABUMIP_m3'].assign_coords({'m': 3}),
                                       GL_flux['flux_Gt_ABUMIP_m5'].assign_coords({'m': 5})], dim='m')

GL_flux_ABUMIP = GL_flux_ag['flux_Gt_ABUMIP']

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
isf_mask = file_isf['ISF_mask']

In [ ]:
file_isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')

In [ ]:
weight_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_withoutGISS.nc')
weight_2300_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_2300_withoutGISS.nc')
file_viability_info = xr.open_dataset(inputpath_weights + 'all_fluxes_br_withoutGISS.nc')

In [ ]:
mask_basins = xr.open_dataset(inputpath_data + 'Mask_Iceshelf_4km_IMBIE_withNisf.nc')

PREPARE DATA

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})

In [ ]:
model_2300 = ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM', 'IPSL-CM6A-LR','MRI-ESM2-0','UKESM1-0-LL'] #,'GISS-E2-1-H'

In [ ]:
bay_weights = weight_file['bay_weights']
bay_weights_2300 = weight_2300_file['bay_weights']

In [ ]:
mass_balance_weighted = file_viability_info['MASS_BALANCE'].weighted(bay_weights * sens_weights)
mass_balance_weighted_2300 = file_viability_info['MASS_BALANCE'].sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [ ]:
weighted_quantiles = mass_balance_weighted.quantile([0,0.1,0.33,0.66,0.5,0.9],dim=['model','param','m'])
unweighted_quantiles = file_viability_info['MASS_BALANCE'].quantile([0,0.1,0.33,0.66,0.5,0.9],dim=['model','param','m'])


In [ ]:
weighted_quantiles_2300 = mass_balance_weighted_2300.quantile([0,0.1,0.33,0.66,0.5,0.9],dim=['model','param','m'])
unweighted_quantiles_2300 = file_viability_info['MASS_BALANCE'].sel(model=model_2300).quantile([0,0.1,0.33,0.66,0.5,0.9],dim=['model','param','m'])


In [ ]:
kisf = 10
scen = 'ssp585'
weight_to_plot = weighted_quantiles_2300.sel(Nisf=kisf)
unweight_to_plot = unweighted_quantiles_2300.sel(Nisf=kisf)


plt.figure()

#for mparam in file_viability_info.param:
#    for m in file_viability_info.m:
#        for mod in file_viability_info.model:
#            plt.plot(file_viability_info['MASS_BALANCE'].sel(param=mparam, Nisf=kisf, scenario=scen, m=m, model=mod), alpha=0.05)
plt.fill_between(x=unweight_to_plot.time,y1=unweight_to_plot.sel(scenario=scen, quantile=0.33), y2=unweight_to_plot.sel(scenario=scen, quantile=0.66),alpha=0.2, color='red')
plt.fill_between(x=weight_to_plot.time,y1=weight_to_plot.sel(scenario=scen, quantile=0.33), y2=weight_to_plot.sel(scenario=scen, quantile=0.66),alpha=0.4, color='red')
plt.axhline(y=-89.2024,color='k')
plt.fill_between(x=unweight_to_plot.time,y1=-89.2024-141,y2=-89.2024+141,alpha=0.2,color='grey')
sns.despine()

In [ ]:
unweighted_quantiles.sel(scenario=scen, quantile=0.33)

In [ ]:
kisf = 10
f = plt.figure()

weight = bay_weights * sens_weights

for i,ddim in enumerate(['param','model','m']): #

    if ddim == 'param':
        weight_ddim = weight.sum('Nisf').stack(stack_dim=['model','m'])
    elif ddim == 'model':
        weight_ddim = weight.sum('Nisf').stack(stack_dim=['param','m'])
    elif ddim == 'm':        
        weight_ddim = weight.sum('Nisf').stack(stack_dim=['param','model'])


    bins = weight_ddim[ddim]
    _, bin_edges = np.histogram(weight_ddim.values, bins=np.arange(len(bins)), weights=weight_ddim.values)

    hist = weight_ddim.sum('stack_dim') # / weight_ddim.sum()

    ax = f.add_subplot(1,3,i+1)
    ax.barh(weight_ddim[ddim], hist,  edgecolor='black') #width=1,
    #ax.set_title('sigma_mod = '+str(mult_fac)+' x std deviation')
#f.suptitle(file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
weight

In [ ]:
kisf = 10
f = plt.figure(figsize=(8.25*3/2,8.25/2))

weight = bay_weights * sens_weights
weight_norm = weight / weight.sum(['param','model','m'])

for i,ddim in enumerate(['param','model','m']): #

    if ddim == 'param':
        weight_ddim = weight_norm.sum('Nisf').stack(stack_dim=['model','m'])
    elif ddim == 'model':
        weight_ddim = weight_norm.sum('Nisf').stack(stack_dim=['param','m'])
    elif ddim == 'm':        
        weight_ddim = weight_norm.sum('Nisf').stack(stack_dim=['param','model'])


    bins = weight_ddim[ddim]
    _, bin_edges = np.histogram(weight_ddim.values, bins=np.arange(len(bins)), weights=weight_ddim.values)

    hist = weight_ddim.sum('stack_dim') # / weight_ddim.sum()

    ax = f.add_subplot(1,3,i+1)
    ax.bar(weight_ddim[ddim], hist,  edgecolor='None') #width=1,
    #ax.set_title('sigma_mod = '+str(mult_fac)+' x std deviation')
#f.suptitle(file_isf['isf_name'].sel(Nisf=kisf).values)
sns.despine()
#plt.tight_layout()
f.savefig(plot_path + 'histo_weights_sum_iceshelves.pdf')

In [ ]:
f = plt.figure(figsize=(8.25*3/2,8.25/2))

weight = bay_weights * sens_weights
weight_norm = weight.sel(Nisf=kisf, model=model_2300) / weight.sel(Nisf=kisf).sum(['param','model','m'])

for i,ddim in enumerate(['param','model','m']): #

    if ddim == 'param':
        weight_ddim = weight_norm.sum('Nisf').stack(stack_dim=['model','m'])
    elif ddim == 'model':
        weight_ddim = weight_norm.sum('Nisf').stack(stack_dim=['param','m'])
    elif ddim == 'm':        
        weight_ddim = weight_norm.sum('Nisf').stack(stack_dim=['param','model'])


    bins = weight_ddim[ddim]
    _, bin_edges = np.histogram(weight_ddim.values, bins=np.arange(len(bins)), weights=weight_ddim.values)

    hist = weight_ddim.sum('stack_dim') # / weight_ddim.sum()

    ax = f.add_subplot(1,3,i+1)
    ax.bar(weight_ddim[ddim], hist,  edgecolor='None')#width=1,
    plt.xticks(rotation=90)
    #ax.set_title('sigma_mod = '+str(mult_fac)+' x std deviation')
#f.suptitle(file_isf['isf_name'].sel(Nisf=kisf).values)
sns.despine()
#plt.tight_layout()
f.savefig(plot_path + 'histo_weights_sum_iceshelve_2300.pdf')

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

In [ ]:
ddim = 'm' #'param','model','m'


f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0

weight = bay_weights * sens_weights

for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    weight_norm_kisf = weight.sel(Nisf=kisf, model=model_2300) / weight.sel(Nisf=kisf).sum(['param','model','m'])

    if ddim == 'param':
        weight_ddim = weight_norm_kisf.stack(stack_dim=['model','m'])
    elif ddim == 'model':
        weight_ddim = weight_norm_kisf.stack(stack_dim=['param','m'])
    elif ddim == 'm':        
        weight_ddim = weight_norm_kisf.stack(stack_dim=['param','model'])

    bins = weight_ddim[ddim]
    _, bin_edges = np.histogram(weight_ddim.values, bins=np.arange(len(bins)), weights=weight_ddim.values)

    hist = weight_ddim.sum('stack_dim') # / weight_ddim.sum()

    ax[i].bar(weight_ddim[ddim], hist,  edgecolor='None')#width=1,

    #ax[i].set_title(ano_choice.sel(Nisf=kisf))

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:#
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))
    #if file_isf['isf_name'].sel(Nisf=kisf).values not in ['Larsen B', 'Wordie']:
    #    ax[i].set_ylim(0,mass_balance_simu_anoISMIP.sel(Nisf=kisf).max().values)
    #ax[i].axvline(x=30, c='k', linestyle='--')
    
    if i < 56:
        ax[i].set_xticklabels('')
    if i not in [0,8,16,24,32,40,48,56]:
        ax[i].set_yticklabels('')
        
    i = i+1

    plt.xticks(rotation=90)
#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'histo_weights_per_iceshelf_2300_'+ddim+'.pdf')